In [1]:
import pandas as pd





In [7]:
url_recursos = 'https://github.com/amiltonmendes/consumo-aparente/raw/refs/heads/main/MIP/68_tab1_2021.xls'
url_usos = 'https://github.com/amiltonmendes/consumo-aparente/raw/refs/heads/main/MIP/68_tab2_2021.xls'


In [28]:
oferta = pd.read_excel(url_recursos, sheet_name='oferta',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
#Renomear as duas primeiras colunas que não foram carregadas por estarem mescladas
oferta= oferta.rename(columns={oferta.columns[0] : 'Código do produto', oferta.columns[1] : 'Descrição do produto'})
#Remover a primeira linha (vazia)
oferta = oferta.iloc[1:]
oferta.head(3)


Index(['Unnamed: 0', 'Unnamed: 1', 'Oferta total\na preço\nde consumidor',
       'Margem \nde\ncomércio', 'Margem\nde\ntransporte',
       'Imposto\nde\nimportação', 'IPI', 'ICMS',
       'Outros impostos menos subsídios',
       'Total de impostos líquidos de subsídios',
       'Oferta total\n a preço básico '],
      dtype='object')


,Código do produto,Descrição do produto,Oferta total\na preço\nde consumidor,Margem \nde\ncomércio,Margem\nde\ntransporte,Imposto\nde\nimportação,IPI,ICMS,Outros impostos menos subsídios,Total de impostos líquidos de subsídios,Oferta total\n a preço básico
1,01911,"Arroz, trigo e outros cereais",50589.0,4691.0,1521.0,2.0,0.0,68.0,550.0,620.0,43757.0
2,01912,Milho em grão,125926.0,12347.0,3147.0,0.0,0.0,471.0,409.0,880.0,109552.0
3,01913,"Algodão herbáceo, outras fibras da lav. tempor...",34790.0,2068.0,436.0,3.0,0.0,90.0,318.0,411.0,31875.0


In [29]:
producao = pd.read_excel(url_recursos, sheet_name='producao',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
producao = producao.rename(columns={producao.columns[0] : 'Código do produto', producao.columns[1] : 'Descrição do produto'})
producao = producao.iloc[1:]
producao.head(3)

,Código do produto,Descrição do produto,"0191\nAgricultura, inclusive o apoio à agricultura e a pós-colheita","0192\nPecuária, inclusive o apoio à pecuária",0280\nProdução florestal; pesca e aquicultura,0580\nExtração de carvão mineral e de minerais não-metálicos,"0680\nExtração de petróleo e gás, inclusive as atividades de apoio","0791\nExtração de minério de ferro, inclusive beneficiamentos e a aglomeração","0792\nExtração de minerais metálicos não-ferrosos, inclusive beneficiamentos","1091\nAbate e produtos de carne, inclusive os produtos do laticínio e da pesca",...,"8000\nAtividades de vigilância, segurança e investigação","8400\nAdministração pública, defesa e seguridade social",8591\nEducação pública,8592\nEducação privada,8691\nSaúde pública,8692\nSaúde privada,"9080\nAtividades artísticas, criativas e de espetáculos",9480\nOrganizações associativas e outros serviços pessoais,9700\nServiços domésticos,Total\ndo produto
1,01911,"Arroz, trigo e outros cereais",31603.0,964.0,18.0,0.0,0.0,0.0,0.0,0.0,...,0.0,91.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32676.0
2,01912,Milho em grão,100288.0,4726.0,159.0,0.0,0.0,0.0,0.0,0.0,...,0.0,64.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,105237.0
3,01913,"Algodão herbáceo, outras fibras da lav. tempor...",31752.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31774.0


In [57]:
importacao = pd.read_excel(url_recursos, sheet_name='importacao',skiprows=3,skipfooter=5,dtype={'Unnamed: 0' : str})
importacao = importacao.rename(columns={importacao.columns[0] : 'Código do produto', importacao.columns[1] : 'Descrição do produto'})
importacao = importacao.iloc[1:]
importacao.tail(3)


,Código do produto,Descrição do produto,Importação de bens\ne serviços (1)
126,94802,"Manutenção de computadores, telefones e objeto...",0.0
127,94803,Serviços pessoais,0.0
128,97001,Serviços domésticos,0.0


In [32]:
exportacao = pd.read_excel(url_usos, sheet_name='demanda',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
exportacao = exportacao.rename(columns={exportacao.columns[0] : 'Código do produto', exportacao.columns[1] : 'Descrição do produto'})
exportacao = exportacao.iloc[1:]
exportacao.head(3)


,Código do produto,Descrição do produto,Exportação\nde bens e\nserviços (1),Consumo\ndo governo,Consumo\ndas\n ISFLSF,Consumo \ndas famílias,Formação bruta\nde capital fixo,Variação\nde estoque,Demanda\nfinal,Demanda\ntotal
1,01911,"Arroz, trigo e outros cereais",1979.0,70.0,0.0,1409.0,0.0,12245.0,15703.0,50589.0
2,01912,Milho em grão,22656.0,53.0,0.0,9779.0,0.0,30117.0,62605.0,125926.0
3,01913,"Algodão herbáceo, outras fibras da lav. tempor...",18609.0,0.0,0.0,11.0,0.0,-830.0,17790.0,34790.0


In [48]:
preco_consumidor,preco_basico = oferta[oferta['Código do produto'].str.startswith('21')][['Oferta total\na preço\nde consumidor','Oferta total\n a preço básico ']].values[0]
fator_ajuste = preco_basico/preco_consumidor
fator_ajuste


np.float64(0.6596922265463513)

In [72]:
valor_producao = producao[producao['Código do produto'].str.startswith("21")][['Total\ndo produto']].values[0][0]
valor_producao

np.float64(108586.0)

In [73]:
valor_exportacao = exportacao[exportacao['Código do produto'].str.startswith("21")][['Exportação\nde bens e\nserviços (1)']].values[0][0]
valor_exportacao = valor_exportacao*fator_ajuste
valor_exportacao


np.float64(3996.415508417796)

In [75]:
valor_importacao = importacao[importacao['Código do produto'].str.startswith("21")]['Importação de bens\ne serviços (1)'].values[0]


In [76]:
total = valor_importacao - valor_exportacao + valor_producao
peso_importacao = valor_importacao/total
peso_exportacao = valor_exportacao/total
peso_producao = valor_producao/total

In [77]:
print(f'Peso da importação: {peso_importacao}')
print(f'Peso da exportação: {peso_exportacao}')
print(f'Peso da produção: {peso_producao}')

Peso da importação: 0.42250981749526956
Peso da exportação: 0.02206616206135398
Peso da produção: 0.5995563445660844


In [87]:
!pip install pyIpeaData

In [88]:
import pyIpeaData as ip

dados_exportacao_farmacos = ip.get_serie('FUNCEX12_XQFARM2N12')
dados_importacao_farmacos = ip.get_serie('FUNCEX12_MQFARM2N12')
dados_importacao_farmacos.tail(3)


,SERCODIGO,VALDATA,VALVALOR,NIVNOME,TERCODIGO
334,FUNCEX12_MQFARM2N12,2024-11-01T00:00:00-03:00,140.18,,
335,FUNCEX12_MQFARM2N12,2024-12-01T00:00:00-03:00,141.96,,
336,FUNCEX12_MQFARM2N12,2025-01-01T00:00:00-03:00,174.28,,


In [85]:
!pip install sidrapy

In [89]:
import requests
import json
from pandas import json_normalize
from datetime import datetime


url = 'https://apisidra.ibge.gov.br/values/t/8888/n1/all/v/12606/p/all/c544/129330/d/v12606%205'
r = requests.get(url)
#Apresentar data de carregamento
#st.write(f"Data de carregamento: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
a = json.loads(r.text)
res = json_normalize(a)

df = pd.DataFrame(res)


In [90]:
df.head()

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Brasil (Código),Brasil,Variável (Código),Variável,Mês (Código),Mês,Seções e atividades industriais (CNAE 2.0) (Có...,Seções e atividades industriais (CNAE 2.0)
1,1,Brasil,30,Número-índice,55.42792,1,Brasil,12606,PIMPF - Número-índice (2022=100),200201,janeiro 2002,129330,3.21 Fabricação de produtos farmoquímicos e fa...
2,1,Brasil,30,Número-índice,76.43190,1,Brasil,12606,PIMPF - Número-índice (2022=100),200202,fevereiro 2002,129330,3.21 Fabricação de produtos farmoquímicos e fa...
3,1,Brasil,30,Número-índice,86.01877,1,Brasil,12606,PIMPF - Número-índice (2022=100),200203,março 2002,129330,3.21 Fabricação de produtos farmoquímicos e fa...
4,1,Brasil,30,Número-índice,91.12036,1,Brasil,12606,PIMPF - Número-índice (2022=100),200204,abril 2002,129330,3.21 Fabricação de produtos farmoquímicos e fa...
